# Avaliação 2

### Questão 1
Utilizando a base Titanic disponível no kaggle (https://www.kaggle.com/c/titanic/data) elabore uma solução utilizando dois algoritmos de aprendizagem de máquina do seu conhecimento para classificar se o passageiro tem ou não chance de sobreviver. Os resultados dessa questão deverão ser descritos detalhadamente no relatório.
* Lembre de fazer todo o pré-processamento, explicando suas decisões;
* Avalie os resultados usando diferentes métricas.
* Teste 3 variações de parâmetros dos métodos;

#### Pré-processamento

É dado início ao pré-processamento, importando as bibliotecas necessárias e instanciando os databases forneceidos através do pandas.

É importante entender que:
* __gs__ equivale à instância do database que contém as saídas (coluna *Survived*) corretas para os testes
* __tr__ equivale à instância do database que contém os treinos
* __ts__ equivale à instância do database que contém os testes

Vale lembrar que o database __gs__ refere-se ao __teste__, e que o __teste__ não pertence ao treino. Portanto será interessante unir os dois databases futuramente.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import cluster, neighbors, svm, metrics, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

gs = pd.read_csv("data/gender_submission.csv",sep=",")
tr = pd.read_csv("data/train.csv",sep=",")
ts = pd.read_csv("data/test.csv",sep=",")

#### Avaliando a quantidade de itens em cada database

Para entender a quantidade de itens com o qual se está trabalhando, é feita uma leitura através do comando __*len()*__.

In [2]:
print("Tamanho do database de treino:", len(tr))
print("Tamanho do database de testes:", len(ts))
print("Tamanho do database que contém as saídas corretas do teste:", len(gs))

Tamanho do database de treino: 891
Tamanho do database de testes: 418
Tamanho do database que contém as saídas corretas do teste: 418


#### Unindo todos os databases

Para passarem pelo pré-processamento, é interessante que todos os dados estejam juntos. Desta forma o __gs__ será unido ao __ts__, que será, por sua vez, unido ao __tr__.

❗️ __Utilizaremos o termo *dfTotal* para se referir ao DataFrame completo, que inclui o teste e o treino.__ ❗️

In [3]:
dfTotal = pd.concat([tr,pd.concat([gs, ts.drop(columns=['PassengerId'])], axis=1)])

#### Analisando quantos itens nulos há no database

É importante conhecer quantos itens nulos existem no database para entender qual deve ser a medida adequada a se tomar mediante a quantidade de itens nulos.

In [4]:
print("-->Nulidade:")
print(dfTotal.isnull().sum())

-->Nulidade:
PassengerId       0
Survived          0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64


⚠️ __Os itens nulos de cabine representam, no treino, mais de 77% do total de informações. Excluir todas essas informações pode representar uma alta perda ao treinamento do IA, ao passo que completar todo esse conteúdo com informações aleatórias ou fixas também podem trazer algum nível de imprecisão aos testes. Desta forma, optou-se por excluir essa coluna.__ ⚠️

#### Excluindo a coluna de Cabine

Como dito acima, a coluna *Cabin* apresenta muitos valores nulos. Desta forma passa a ser interessante excluir essa coluna.

In [5]:
dfTotal = dfTotal.drop(columns=['Cabin'])

#### Analisando quantos itens nulos há no database

Ainda analisando a quantidade de elementos nulos, será feita a análise novamente agora sem a coluna de cabine, que previamente apresentou várias informações faltantes.

In [7]:
print("-->Nulidade:")
print(dfTotal.isnull().sum())

-->Nulidade:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            263
SibSp            0
Parch            0
Ticket           0
Fare             1
Embarked         2
dtype: int64


❗️ __Considerando que *Fare* (Tarifa) e *Embarked* apresentam uma quantidade ínfima de valores nulos, considerou-se interessante, na coluna Fare, considerar a mediana no lugar do valor nulo e, no Embarked, o uso da moda dos valores.__ ❗️

#### Substituindo valores de Fare, Age e Embarked

Será considerada a moda dos valores de __Embarked__ para substituir os valores nulos e a mediana de __Fare__ e __Age__ para substituir os valores faltantes.

Note que, por mais que haja uma quantidade mais expressiva de valores nulos para *Age*, isso representa apenas 20%, da quantidade total. Sendo assim é entendeu-se como viável substituir os valores de idade pela mediana de todas as idades.

In [9]:
dfTotal["Fare"].fillna(dfTotal["Fare"].median(), inplace=True)
print(dfTotal.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            263
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64


In [13]:
dfTotal["Embarked"].fillna(dfTotal["Embarked"].mode()[0],inplace=True)
print(dfTotal.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            263
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         0
dtype: int64


In [14]:
dfTotal["Age"].fillna(dfTotal["Age"].median(),inplace=True)
print(dfTotal.isnull().sum())

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64
